In [1]:
#!pip install map_boxes
#!pip install tqdm
#!pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
def load_data():
    TEST_JSON = './dataset/test.json'
    TRAIN_JSON = './dataset/train.json'
    PRED_CSV = './output_test.csv'

    # load Test json
    with open(TEST_JSON, 'r') as outfile:
        test_json = (json.load(outfile))
    # load Train json
    with open(TRAIN_JSON, 'r') as outfile:
        train_json = (json.load(outfile))

    # load prediction
    pred_df = pd.read_csv(PRED_CSV)
    
    return test_json, train_json, pred_df

In [10]:
def create_json(filename, option):
    # init json, csv
    test_json, train_json, pred_df = load_data()
    
    result = {}

    result['info'] = train_json['info']
    result['licenses'] = train_json['licenses']
    result['images'] = train_json['images']
    
    train_len = len(train_json['images'])
    
    # append test image & change test image id
    for test_img in test_json['images']:
        test_img['id'] += train_len
        result['images'].append(test_img)
    
    result['categories'] = train_json['categories']
    
    templist = []
    templist = train_json['annotations']
    
    bbox_id = len(train_json['annotations'])
    for idx, item in enumerate(pred_df['PredictionString']):
        # exception >> if item == nan
        if type(item) != str:
            continue
            
        for v in np.array(item.strip().split(' ')).reshape(-1,6):
            # check category option
            if int(v[0]) not in option:
                continue
            tempdict = {}
            tempdict['image_id'] = idx + train_len
            tempdict['category_id'] = int(v[0])
            tempdict['area'] = float(v[4])*float(v[5])
            tempdict['bbox'] = [float(v[2]),float(v[3]), float(v[4]),float(v[5])]
            tempdict['iscrowd'] = 0
            tempdict['id'] = bbox_id

            templist.append(tempdict)
            bbox_id = bbox_id+1
            
    result['annotations'] = templist

    with open(f'./dataset/{filename}.json', 'w') as outfile:
        json.dump(result, outfile)

In [11]:
# filename, option >> array for selecting categories ex) [1,6,7,8]
create_json('pseudo_labeled', [0,1,2,3,4,5,6,7,8,9])